# Extract Features with ResNet

In [ ]:
# !pip3 install datasets

In [1]:
import os
import sys
import csv
import pickle
import torch

import pandas as pd
import numpy as np

from transformers import AutoImageProcessor, ResNetModel
from datasets import load_dataset
from PIL import Image

# Load Data

In [2]:
BASE = '/Users/brinkley97/Documents/development/'
CLASS_PATH = 'classes/csci_535_multimodal_probabilistic_learning/'
DATASET_PATH = 'datasets/project/'

In [31]:
def load_data(file):
    original_data = pd.read_csv(file)
    # original_data = pd.DataFrame(file)
    copy_of_data = original_data.copy()
    return copy_of_data

In [32]:
file_paths_crema_d = BASE + CLASS_PATH + DATASET_PATH + 'crema_d/all_paths_to_crema_d_faces.zip'
file_paths_msp_t = BASE + CLASS_PATH + DATASET_PATH + 'msp_t/all_paths_to_msp_t_faces.zip'

In [33]:
dataset_paths_copy_crema_d = load_data(file_paths_crema_d)

In [38]:
dataset_paths_copy_crema_d

Unnamed: 0                                            crema_d
0              0  /Users/brinkley97/Documents/development/classe...
1              1  /Users/brinkley97/Documents/development/classe...
2              2  /Users/brinkley97/Documents/development/classe...
3              3  /Users/brinkley97/Documents/development/classe...
4              4  /Users/brinkley97/Documents/development/classe...
...          ...                                                ...
7437        7437  /Users/brinkley97/Documents/development/classe...
7438        7438  /Users/brinkley97/Documents/development/classe...
7439        7439  /Users/brinkley97/Documents/development/classe...
7440        7440  /Users/brinkley97/Documents/development/classe...
7441        7441  /Users/brinkley97/Documents/development/classe...

[7442 rows x 2 columns]

In [ ]:
dataset_paths_copy_msp_t = load_data(file_paths_msp_t)

In [ ]:
dataset_paths_copy_msp_t

# ResNet

In [3]:
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetModel.from_pretrained("microsoft/resnet-50")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of the model checkpoint at microsoft/resnet-50 were not used when initializing ResNetModel: ['classifier.1.weight', 'classifier.1.bias']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Extract Features

In [4]:
def write_list(list_to_save, file):
    
    # store list in binary file so 'wb' mode
    with open(file, 'wb') as fp:
        pickle.dump(list_to_save, fp)
        # print('Done writing list into a binary file')

In [5]:
def extract_features(path_to_folders, store_extracted_features_file):
    """
    """
    len(path_to_folders)
    
    store_extracted_features_msp_t = []
    store_folder_extracted_features = []

    for path_to_folders_idx in range(len(path_to_folders)):
        path_to_face_folder = path_to_folders[path_to_folders_idx]
        print(path_to_folders_idx, path_to_face_folder)
        
        single_folder_all_faces = os.listdir(path_to_face_folder)
        
       
        for single_folder_all_faces_idx in range(len(single_folder_all_faces)):
            image_name = single_folder_all_faces[single_folder_all_faces_idx]

            path_to_image = path_to_face_folder + image_name
            image = Image.open(path_to_image)

            # inputs = feature_extractor(images=image, return_tensors="pt")
            # outputs = model(**inputs)
            # last_hidden_states = outputs.last_hidden_state 
            
            inputs = image_processor(image, return_tensors="pt")
            
            with torch.no_grad():
                outputs = model(**inputs)

            last_hidden_states = outputs.last_hidden_state
            
            
            store_folder_extracted_features.append(last_hidden_states)
            
        store_extracted_features_msp_t.append(store_folder_extracted_features)
        write_list(store_extracted_features_msp_t, store_extracted_features_file)
    
    return store_extracted_features_msp_t

In [8]:
save_resnet_extracted_features_crema_d = BASE + CLASS_PATH + DATASET_PATH + 'crema_d/extracted_features_crema_d/save_resnet_extracted_features_crema_d'
save_resnet_extracted_features_crema_d

'/Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/project/crema_d/extracted_features_crema_d/save_resnet_extracted_features_crema_d'

In [44]:
crema_d_faces = dataset_paths_copy_crema_d['crema_d']
list_crema_d_faces = list(crema_d_faces)
# list_crema_d_faces

In [ ]:
# extracted_features_crema_d = extract_features(list_crema_d_faces, save_resnet_extracted_features_crema_d)

In [22]:
subset = BASE + CLASS_PATH + DATASET_PATH + 'crema_d/subset_frames_to_faces_crema_d_sub'
subset_paths = os.listdir(subset)
subset_paths.sort()
type(subset_paths), len(subset_paths)
# subset_paths

(list, 6150)

In [48]:
build_subset_paths = []
for idx in range(len(subset_paths)):
    folder = subset_paths[idx]
    # print(folder)
    
    build_path = subset + '/' + folder + '/'
    
    build_subset_paths.append(build_path)
    
    # print()

In [51]:
len(build_subset_paths)

6150

In [53]:
# build_subset_paths

In [60]:
extracted_features_crema_d = extract_features(build_subset_paths, save_resnet_extracted_features_crema_d)

0 /Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/project/crema_d/subset_frames_to_faces_crema_d_sub/1001_DFA_ANG_XX_frames_to_cropped_face_subset/
1 /Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/project/crema_d/subset_frames_to_faces_crema_d_sub/1001_DFA_DIS_XX_frames_to_cropped_face_subset/
2 /Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/project/crema_d/subset_frames_to_faces_crema_d_sub/1001_DFA_FEA_XX_frames_to_cropped_face_subset/
3 /Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/project/crema_d/subset_frames_to_faces_crema_d_sub/1001_DFA_HAP_XX_frames_to_cropped_face_subset/
4 /Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/project/crema_d/subset_frames_to_faces_crema_d_sub/1001_DFA_NEU_XX_frames_to_cropped_face_subset/


In [61]:
save_1290_resnet_extracted_features_crema_d = BASE + CLASS_PATH + DATASET_PATH + 'crema_d/extracted_features_crema_d/save_1290_resnet_extracted_features_crema_d'
save_1290_resnet_extracted_features_crema_d

save_resnet_extracted_features_crema_d = BASE + CLASS_PATH + DATASET_PATH + 'crema_d/extracted_features_crema_d/save_resnet_extracted_features_crema_d'
save_resnet_extracted_features_crema_d

'/Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/project/crema_d/extracted_features_crema_d/save_resnet_extracted_features_crema_d'

In [62]:
# Read list to memory
def read_list(list_of_features):
    # for reading also binary mode is important
    with open(list_of_features, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

In [63]:
# read_resnet_crema_d = read_list(save_1290_resnet_extracted_features_crema_d)
read_resnet_crema_d = read_list(save_resnet_extracted_features_crema_d)

In [64]:
len(read_resnet_crema_d), read_resnet_crema_d[0][0].shape

(6150, torch.Size([1, 2048, 7, 7]))

# MSP T

In [ ]:
save_resnet_extracted_features_msp_t = BASE + CLASS_PATH + DATASET_PATH + 'msp_t/extracted_features_resnet_msp_t/save_resnet_extracted_features_msp_t'
save_resnet_extracted_features_msp_t

In [ ]:
msp_t_faces = dataset_paths_copy_msp_t['msp_t']
list_msp_t_faces = list(msp_t_faces)
# list_msp_t_faces

In [ ]:
extracted_features_msp_t = extract_features(list_msp_t_faces, save_resnet_extracted_features_msp_t)

In [ ]:
len(extracted_features_msp_t)

In [ ]:
len(extracted_features_msp_t[0]), len(extracted_features_msp_t[1])

In [ ]:
# extracted_features_msp_t[0][4].shape, extracted_features_msp_t[0][4].shape, extracted_features_msp_t[0][4].shape, extracted_features_msp_t[0][4].shape

In [ ]:
# Read list to memory
def read_list(list_of_features):
    # for reading also binary mode is important
    with open(list_of_features, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

In [ ]:
reading = read_list(save_resnet_extracted_features_msp_t)

In [ ]:
len(reading)

In [ ]:
reading[651][0].shape

In [ ]:
reading = read_list(save_vit_extracted_features_msp_t2)

In [ ]:
stop timer